## Python3 Generating Test Results For Reproducibiity Study:

## Journal PLOS ONE Extracting Schemas from Thought Records [paper](https://doi.org/10.1371/journal.pone.0257832)

<br>
The purpose of this script is to generate the results using the models supporting Hypothesis 1 of the study. 
The wall time of runtimes are provided in the first comment of cells of time intensive code. Additionally, the cell magic "%%time" in these cells ensures that runtimes are printed so that these can be compared to the reported runtimes to get appropriate estimates when running on a different machine.<br>

<br>
The modules below need to be installed before running the code:
    <ol>
    <li>numpy==1.21.6</li>
    <li>pandas==1.3.5</li>
    <li>gensim==3.6.0</li>
    <li>statsmodels==0.10.2</li>
    <li>rasa-nlu</li>
    <li>tensorflow==2.8.0</li>
    <li>talos</li>
    <li>scipy==1.4.1</li>
    <li>scikit-learn==0.23.2</li>
    </ol>

<br>
The following inputs are required and can be found in the group165/Data2 directory:
    <ol>
    <li>glove.6B directory</li>
    <li>DatasetsForH1 directory</li>
    <li>mlm_rnn.zip file</li>
    <li>MLMs directory</li>
    <li>PSMs directory</li>
    </ol>

We import the following packages and functions:

In [2]:
#set seed
seed = 57839
import os
os.environ['PYTHONHASHSEED']=str(seed)
import sys

import random
random.seed(seed)

import numpy as np
np.random.seed(seed)

import csv
import pandas as pd
import scipy
import scipy.stats as stats
import functools
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument


import sklearn
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier, KNeighborsRegressor
from sklearn import metrics, preprocessing, svm
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
import statsmodels.formula.api as smf

import tensorflow as tf
tf.random.set_seed(seed)

from tensorflow.python.keras.metrics import Metric
from tensorflow import keras
import talos
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Flatten, Embedding, SimpleRNN, LSTM, GRU, Bidirectional,Dropout

from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)



2022-05-05 10:32:41.253436: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
print(sys.version)

3.7.13 (default, Mar 28 2022, 07:24:34) 
[Clang 12.0.0 ]


In [6]:
#list packages and their version numbers as used in this script (code is taken from 
#https://stackoverflow.com/questions/40428931/package-for-listing-version-of-packages-used-in-a-jupyter-notebook)
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to add
        # exceptions to this list manually!
        poorly_named_packages = {
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

gensim==3.6.0
keras==2.8.0
tensorflow==2.8.0
numpy==1.21.6
scipy==1.4.1
pandas==1.3.5
talos==1.2.3
scikit-learn==0.23.2
statsmodels==0.13.2


> We also set the working directory:

In [7]:
os.chdir("/Users/sampathg/Documents/MyUIUC/CS598-DLHC/group165")

## Importing the datasets from csv
> The preprocessed utterances are split into three sets in the R script. They are saved in three separate csv files. Additionally, the manually assigned labels that correspond with the utterances are saved in three separate csv files.

In [8]:
# read in datasets (already pre-processed)
def readcsv(fname,istext):
    if istext:
        with open(fname,'rt') as f:
            reader=csv.reader(f)
            next(reader)
            data = []
            for row in reader:
                for item in row:
                    data.append(item)
            f.close()
    else:
        with open(fname,'r') as f:
            reader=csv.reader(f,delimiter=';')
            next(reader)
            data = list(reader)
            data = np.asarray(data, dtype='int')
            f.close()
    return data 

# read in training, validation, and test set utterances
train_text = readcsv('Data2/DatasetsForH1/H1_train_texts.csv',True)
val_text = readcsv('Data2/DatasetsForH1/H1_validate_texts.csv', True)
test_text = readcsv('Data2/DatasetsForH1/H1_test_texts.csv',True)

# read in training, validation, and test set labels
train_labels = readcsv('Data2/DatasetsForH1/H1_train_labels.csv',False)[:,0:9]
val_labels = readcsv('Data2/DatasetsForH1/H1_validate_labels.csv', False)[:,0:9]
test_labels = readcsv('Data2/DatasetsForH1/H1_test_labels.csv',False)[:,0:9]

In [9]:
print(train_text[0:5])

['lot people may think well lot people might not like me', 'might not working fast enough their standards', 'may not able graduate', 'would get bad performance review', 'friends will get annoyed by me']


In [10]:
print(train_labels[0:5,:])

[[2 0 0 0 0 0 0 0 3]
 [0 3 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 0 3]]


In [11]:
print(test_labels[0:5,:])

[[0 3 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0]
 [0 3 0 0 0 0 0 0 0]
 [0 0 3 0 0 0 0 0 0]
 [2 0 2 0 0 0 0 0 0]]


> As can be seen, some utterances have multiple schemas assigned. However, overall, the label matrices are sparse matrices. The first column of the labels corresponds to the "Attachment" schema, the second to the "Competence" schema, the third to last to the "Other's views on self" schema.

In [12]:
#for later use
schemas = ["Attach","Comp","Global","Health","Control","MetaCog","Others","Hopeless","OthViews"]

## Embedding the utterances using GLoVE
> We have opted for representing the words in utterances as word vectors. We adopt the GLoVE word vector space that has been created with Wikipedia 2014. First, we tokenize the top 2000 words of the training set.  

In [13]:
# prepare tokenizer
max_words = 2000
t = Tokenizer(num_words = max_words)
t.fit_on_texts(train_text)
vocab_size = len(t.word_index) + 1
print(vocab_size)

2624


> The tokenizer takes the words and indexes these based on frequency. For the recurrent neural net, we need padded utterances sequences. Texts_to_sequences simply represents each utterance as a vector of tokens. Padding ensures that all vectors are of the same length, by appending 0s to the end of shorter vectors. We pad to a length of 25 words.

In [14]:
# integer encode all utterances
encoded_train = t.texts_to_sequences(train_text)
encoded_validate = t.texts_to_sequences(val_text)
encoded_test = t.texts_to_sequences(test_text)

# pad documents to a max length of 25 words
max_length = 25

padded_train = pad_sequences(encoded_train, maxlen=max_length, padding='post')
padded_validate = pad_sequences(encoded_validate, maxlen=max_length, padding='post')
padded_test = pad_sequences(encoded_test, maxlen=max_length, padding='post')

print(encoded_train[0:5])

[[147, 28, 48, 37, 101, 147, 28, 32, 1, 8, 5], [32, 1, 155, 658, 14, 125, 568], [48, 1, 19, 448], [2, 11, 53, 449, 659], [50, 6, 11, 373, 98, 5]]


In [15]:
print(padded_train[0:5])

[[147  28  48  37 101 147  28  32   1   8   5   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 32   1 155 658  14 125 568   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 48   1  19 448   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  2  11  53 449 659   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [ 50   6  11 373  98   5   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]]


> We can now load the GLoVE embeddings into memory.

In [16]:
%%time
# wall time to run: ~ 10sec
# load all embeddings into memory
embeddings_index = dict()
f = open('Data2/glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.
CPU times: user 8.15 s, sys: 322 ms, total: 8.47 s
Wall time: 8.55 s


> We can then create an embedding matrix by taking each word of the training set and finding the corresponding word vector in the GLoVE data. We only work with 100 dimensional representations.

In [18]:
vec_dims = 100
embedding_matrix = np.zeros((vocab_size, vec_dims))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [19]:
# create tfidf weighted encoding matrix of utterances
train_sequences = t.texts_to_matrix(train_text,mode='tfidf')
val_sequences =  t.texts_to_matrix(val_text,mode='tfidf')
test_sequences = t.texts_to_matrix(test_text,mode='tfidf')
print(train_sequences[0:5])
print(train_sequences.shape)

[[0.         1.29214445 0.         ... 0.         0.         0.        ]
 [0.         1.29214445 0.         ... 0.         0.         0.        ]
 [0.         1.29214445 0.         ... 0.         0.         0.        ]
 [0.         0.         1.69021763 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
(4151, 2000)


In [20]:
# we want to normalize the word vectors
def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
       return v
    return v / norm

In [21]:
# create utterance embeddings as tfidf weighted average of normalized word vectors
def seq2vec(datarow,embedmat):
  #initialize an empty utterance vector of the same length as word2vec vectors
  seqvec = np.zeros((100,))
  #counter for number of words in a specific utterance
  wordcount = 1
  #we iterate over the 2000 possible words in a given utterance
  wordind = 1
  while (wordind < len(datarow)):
    #the tf-idf weight is saved in the cells of datarow
    tfidfweight = datarow[wordind]
    if not tfidfweight is None:
      wordembed = tfidfweight * embedmat[wordind,]
      seqvec = seqvec + normalize(wordembed)
      wordcount = wordcount + 1
    wordind = wordind + 1
  return seqvec/wordcount

In [22]:
# go through the matrix and embed each utterances
def embed_utts(sequences,embedmat):
  vecseq = [seq2vec(seq,embedmat)for seq in sequences]
  return vecseq

> we now have everything needed to create the utterance embeddings

In [23]:
%%time
# wall time to run: ~ 1min 14s
# embedd all three datasets
train_embedutts = embed_utts(train_sequences,embedding_matrix)
val_embedutts = embed_utts(val_sequences,embedding_matrix)
test_embedutts = embed_utts(test_sequences,embedding_matrix)
print(train_embedutts[0])

[-3.44478099e-05  3.48539840e-04  3.35962753e-04 -3.70855457e-04
 -2.63147438e-04  1.07227074e-04 -1.66559109e-04  1.94234500e-05
  7.42420570e-05 -1.80615841e-04  1.80387578e-05  4.92242034e-05
  2.75006568e-04  2.34416192e-05  8.31148165e-05 -2.93833280e-04
 -7.15121389e-05  2.98592314e-04 -4.55134987e-04  4.72657153e-04
  2.57585086e-04  1.69741478e-04  7.75960265e-05 -2.15817394e-04
 -4.34789085e-05  7.24571212e-05 -1.54585404e-04 -4.98166781e-04
  1.93941088e-04 -1.74921206e-04  2.37557331e-05  4.85150809e-04
  3.08554881e-05 -4.62293641e-05  1.35110613e-04  2.80284189e-04
 -3.22980711e-05  3.12968134e-04  8.27704500e-05 -2.40951546e-04
 -3.13527886e-04 -1.35440392e-04 -2.05195768e-05 -4.81099111e-04
 -2.75375333e-04 -1.27601856e-04  2.50256011e-04 -2.50631136e-04
 -1.51297680e-04 -8.33555219e-04  3.54382525e-05 -8.74190709e-05
  1.05239327e-05  8.00132559e-04 -1.52039351e-04 -1.90058573e-03
  9.49153013e-05 -1.17238522e-05  1.18845110e-03  3.93093667e-04
 -1.88908628e-04  9.94003

## Model Evaluation
> We use the Spearman correlation to evaluate the models and choose the best one, because it can be used for both the regression and the classification outcomes. This is not the case for a weighted Cohen's Kappa, for example, which only works for class labels.

In [24]:
#### Goodness of Fit
def gof_spear(X,Y):
    #spearman correlation of columns (schemas)
    gof_spear = np.zeros(X.shape[1])    
    for schema in range(9):
        rho,p = scipy.stats.spearmanr(X[:,schema],Y[:,schema])
        gof_spear[schema]=rho
    return gof_spear

## k-nearest Neighbors Classification and Regression
> Since we have ordinal labels for our data, we train both classification and regression algorithms and see which one performs better. We also have multi-label data, and therefore write a custom kNN algorithm. We use the cosine distance to find the nearest neighbors.

In [25]:
# cosine distance
def cosine_dist(X,Y):
    return scipy.spatial.distance.cosine(X,Y)

In [26]:
#kNN algorithm
def knn_custom(train_X,test_X,train_y,test_y,k,dist,classification):
    #empty array to collect the results (should have shape of samples to classify)
    votes = np.zeros(test_y.shape)
    #fit the knn
    knn=NearestNeighbors(k, metric=dist)
    knn.fit(train_X)
    #collect neighbors
    i=0 # index to collect votes of the neighbors
    for sample in test_X:
        neighbors=knn.kneighbors([sample],k,return_distance=False)[0]
        if classification:
            output_y = np.zeros((k,test_y.shape[1]))
            j=0
            for neighbor in neighbors:
                output_y[j,:] = train_y[neighbor,:]
                j=j+1
            votes[i,:] = stats.mode(output_y,nan_policy='omit')[0]
        else:
            output_y = np.zeros(test_y.shape[1])
            for neighbor in neighbors:
                output_y += train_y[neighbor,:]
                votes[i,:] = np.divide(output_y,k)
        i=i+1
    return votes

In [27]:
def my_kNN(test_X,test_y,classification):
    if classification:
        my_knn=knn_custom(train_embedutts,test_X,train_labels,test_y,4,cosine_dist,1)
    else:
        my_knn=knn_custom(train_embedutts,test_X,train_labels,test_y,5,cosine_dist,0)
    return gof_spear(my_knn,test_y)

In [28]:
%%time
#wall time to run: ~ 4min
output_kNN_class = my_kNN(test_embedutts,test_labels,1)
output_kNN_reg = my_kNN(test_embedutts,test_labels,0)

CPU times: user 5min 33s, sys: 3.45 s, total: 5min 36s
Wall time: 5min 43s


In [29]:
print('KNN Classification Prediction')
print(pd.DataFrame(data=output_kNN_class,index=schemas,columns=['estimate']))

KNN Classification Prediction
          estimate
Attach    0.550705
Comp      0.690230
Global    0.401123
Health    0.742217
Control   0.107526
MetaCog        NaN
Others    0.279105
Hopeless  0.484137
OthViews  0.454565


In [30]:
print('KNN Regression Prediction')
print(pd.DataFrame(data=output_kNN_reg,index=schemas,columns=['estimate']))

KNN Regression Prediction
          estimate
Attach    0.626743
Comp      0.663091
Global    0.411444
Health    0.534902
Control   0.231541
MetaCog   0.104785
Others    0.243713
Hopeless  0.513825
OthViews  0.458473


## Support vector machine
> The second algorithm we chose are support vector machines (SVMs). Again, we train both a support vector classification (SVC) and a support vectore regression (SVR). We only try all three types of standard kernels and do not do any additional parameter tuning. Just like the kNN, the support vector machine takes as input the utterances encoded as averages of word vectors. Support vector classification and regression do not allow for multilabel output. We therefore train disjoint models, one for each schema.<br>
For both types of SVM, we first transform the input texts as the algorithm expects normally distributed input centered around 0 and with a standard deviation of 1.

In [31]:
#SVM/SVR
def svm_scaler(train_X):
        #scale the data
        scaler_texts = StandardScaler()
        scaler_texts = scaler_texts.fit(train_X)
        return scaler_texts

scaler_texts = svm_scaler(train_embedutts)

>Since SVMs, unlike kNNs, can be trained and reused, we write a method that returns all 9 models and a separate one for the predictions.

In [32]:
def svm_custom(train_X,train_y,text_scaler,kern,classification):
        models=[]
        train_X = text_scaler.transform(train_X)
        #fit a new support vector regression for each schema
        for schema in range(9):
            if classification:
                model = svm.SVC(kernel=kern)
            else:
                model = svm.SVR(kernel=kern)
            model.fit(train_X, train_y[:,schema])
            models.append(model)
        return models

In [33]:
# weighting model output (spearman correlations) by schema frequencies in training set and returning mean over schemas
def performance(train_y,output):
    train_y = np.array(train_y)
    train_y[train_y>0]=1
    weighting = train_y.sum(axis=0)/train_y.shape[0]
    perf = output * weighting
    return np.nanmean(np.array(perf), axis=0)

In [34]:
def svm_predict(svm_models,test_X,train_y,test_y,text_scaler):
    #empty array to collect the results (should have shape of samples to classify)
    votes = np.zeros(test_y.shape)
    for schema in range(9):
        svm_model=svm_models[schema]
        prediction = svm_model.predict(text_scaler.transform(test_X))
        votes[:,schema] = prediction
    out = votes
    gof = gof_spear(out,test_y)
    perf = performance(train_y,gof)
    return out,perf

In [35]:
%%time
# wall time to run: ~ 45sec
# svm
svm_rbf_models =  svm_custom(train_embedutts,train_labels,scaler_texts,'rbf',1)
svm_rbf_out, svm_rbf_perf = svm_predict(svm_rbf_models,val_embedutts,train_labels,val_labels,scaler_texts)

CPU times: user 13.6 s, sys: 147 ms, total: 13.8 s
Wall time: 14.3 s


In [36]:
%%time
# wall time to run: ~ 2min 20sec
# svr
svr_rbf_models =  svm_custom(train_embedutts,train_labels,scaler_texts,'rbf',0)
svr_rbf_out, svr_rbf_perf = svm_predict(svr_rbf_models,val_embedutts,train_labels,val_labels,scaler_texts)

CPU times: user 13.1 s, sys: 143 ms, total: 13.3 s
Wall time: 13.6 s


In [37]:
%%time
# wall time to run: 4sec
def my_svm(test_X,test_y,classification):
    if classification:
        my_svm_out, my_svm_perf=svm_predict(svm_rbf_models,test_X,train_labels,test_y,scaler_texts)
    else:
        my_svm_out, my_svm_perf=svm_predict(svr_rbf_models,test_X,train_labels,test_y,scaler_texts)
    return gof_spear(my_svm_out,test_y)

output_SVC = my_svm(test_embedutts,test_labels,1)
output_SVR = my_svm(test_embedutts,test_labels,0)

CPU times: user 3.6 s, sys: 33.8 ms, total: 3.63 s
Wall time: 3.71 s


In [38]:
print('SVM Classification Prediction')
print(pd.DataFrame(data=output_SVC,index=schemas,columns=['estimate']))

SVM Classification Prediction
          estimate
Attach    0.647714
Comp      0.684661
Global    0.357601
Health    0.729181
Control        NaN
MetaCog        NaN
Others         NaN
Hopeless  0.489903
OthViews  0.476297


In [39]:
print('SVM Regression Prediction')
print(pd.DataFrame(data=output_SVR,index=schemas,columns=['estimate']))

SVM Regression Prediction
          estimate
Attach    0.675340
Comp      0.640866
Global    0.489372
Health    0.349064
Control   0.310007
MetaCog   0.114894
Others    0.185827
Hopeless  0.535979
OthViews  0.516635


## Recurrent neural networks

> We train two types of recurrent neural networks: a multilabel RNN that predicts all 9 schemas simultaneously and a set of 9 single-label RNNs that predict the labels for each schema separately. Each RNN consists of 4 layers: an embedding layer, a bidirectional LSTM layer, a dropout layer, and an output layer.

In [40]:
#we restore the deployed Talos experiment
restore = talos.Restore('Data2/mlm_rnn.zip')  # Changed from /Data/mlm_rnn.zip
#to get the best performing parameters, we get the results of the Talos experiment
scan_results = restore.results

In [41]:
#select the row with the smallest mean absolute error
print(scan_results[scan_results.mean_absolute_error == scan_results.mean_absolute_error.min()]) 

   round_epochs     loss  mean_absolute_error  val_loss  \
7           100  1.26202             0.212742       0.0   

   val_mean_absolute_error  batch_size  dropout  epochs  \
7                      0.0          32      0.1     100   

                     losses  lstm_units optimizer  
7  categorical_crossentropy         100      Adam  


>We have learned that despite setting the random seed values for numpy and tensorflow, some variability remains with each training of the RNNs and our results will therefore not be 100% reproducible. To ensure that we cannot be accused of reporting just a "lucky shot", we have decided to follow the advice given in this blog post https://machinelearningmastery.com/reproducible-results-neural-networks-keras/ . We therefore train 30 multi-label neural nets with the best parameters identfied with the Talos scan. We report the mean Spearman correlations in the article. We do the same with the per-schema RNNs below.

In [42]:
 #generate predictions with the per-schema models
def predict_schema_mlm(test_text, test_labels,fixed=None):
    if fixed is None:
        all_preds = np.zeros((test_labels.shape[0],test_labels.shape[1],30)) 
        all_gofs = np.zeros((30,9)) 
        for j in range(30): 
            model_name = "Data2/MLMs/mlm_" + str(j)
            model = keras.models.load_model(model_name + '.h5')
            preds = model.predict(test_text)
            gofs = gof_spear(preds,test_labels)
            all_preds[:,:,j] = preds
            all_gofs[j,:] = gofs
    else:
        model_name = "Data2/MLMs/mlm_" + str(fixed)
        model = keras.models.load_model(model_name + '.h5')
        all_preds = model.predict(test_text)
        all_gofs = gof_spear(all_preds,test_labels)
    return all_gofs,all_preds

### Training Per-Schema RNNs
> We also train separate RNNs per schema. For this, we can use the output layer to compute a probability for each of the four possible labels. This way, the labels are treated as separate classes. We take over the parameter values from the multilabel model for the number of LSTM units, the dropout rate, the loss function, the evaluation metric, the batch size, and the number of epochs. To obtain the probability for each class, the units of the output layer have a softmax activation function. For the evaluation, the class with the highest probability is chosen per model. The resulting models are written to files and loaded again for prediction.

In [43]:
#define separate models
def perschema_models(train_X, train_y, test_X, test_y):
    model = Sequential()
    e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)
    model.add(e)
    model.add(Bidirectional(LSTM(100)))
    model.add(Dropout(0.1))
    model.add(Dense(4, activation='softmax'))
    # compile the model
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['mean_absolute_error'])
    # summarize the model
    print(model.summary())
    # fit the model
    model.fit(train_X, train_y,
              validation_data=[test_X,test_y],
              batch_size=32, 
              epochs=100, 
              verbose=0)
    out=model.predict(test_X)
    gof,p=scipy.stats.spearmanr(out,test_y,axis=None)
    return gof, model

In [44]:
#load single models
def load_single_models(directory):
    single_models = []
    for i in range(9):
        model_name ='/schema_model_' + schemas[i]
        get_from = directory + model_name
        model = keras.models.load_model(get_from + '.h5')
        single_models.append(model)
    return single_models

In [45]:
#generate predictions with the per-schema models
def predict_schema_psm(test_text, test_labels,fixed=None):
    if fixed is None:
        all_preds = np.zeros((test_labels.shape[0],test_labels.shape[1],30)) 
        all_gofs = np.zeros((30,9)) 
        for j in range(30): 
            directory_name = "Data2/PSMs/per_schema_models_" + str(j)
            preds = np.zeros(test_labels.shape)
            gofs=[]
            single_models = load_single_models(directory_name)
            for i in range(9):
                model = single_models[i]
                out = model.predict(test_text)
                out = out.argmax(axis=1)
                preds[:,i] = out
                gof,p=scipy.stats.spearmanr(out,test_labels[:,i])
                gofs.append(gof)
            all_preds[:,:,j] = preds
            all_gofs[j,:] = gofs
    else:
        directory_name= "Data2/PSMs/per_schema_models_" + str(fixed)
        all_preds = np.zeros(test_labels.shape)
        all_gofs = []
        single_models = load_single_models(directory_name)
        for i in range(9):
            model = single_models[i]
            out = model.predict(test_text)
            out = out.argmax(axis=1)
            all_preds[:,i] = out
            gof,p=scipy.stats.spearmanr(out,test_labels[:,i])
            all_gofs.append(gof)
    return all_gofs,all_preds    

### Generate Testset Predictions with the RNN Models

In [46]:
def my_rnn(test_X,test_y,single):
    if single:
        gof,preds=predict_schema_psm(test_X,test_y)
    else:
        gof,preds=predict_schema_mlm(test_X,test_y)
    
    #make a sum of all classification values
    gof_sum = np.sum(gof,axis=1)
    #sort sums
    gof_sum_sorted = np.sort(gof_sum)
    #pick element that is closest but larger than median (we have even number of elements)
    get_med_element = gof_sum_sorted[15] 
    #get index of median
    gof_sum_med_idx = np.where(gof_sum==get_med_element)[0]
    #choose this as the final model to use in H2 and to report in the paper
    gof_out = gof[gof_sum_med_idx]
    return np.transpose(gof_out),gof_sum_med_idx

In [47]:
%%time
# wall time to run: ~ 6min
# predicting testset with multilabel model
output_mlm,idx_mlm = my_rnn(padded_test,test_labels,0)
# predicting testset with perschema models
output_psm,idx_psm = my_rnn(padded_test,test_labels,1)

CPU times: user 11min 31s, sys: 40.6 s, total: 12min 12s
Wall time: 9min 55s


In [48]:
print('RNN Multilabel Model Testset Output')
print(pd.DataFrame(data=output_mlm,index=schemas,columns=['estimate']))

RNN Multilabel Model Testset Output
          estimate
Attach    0.686899
Comp      0.662779
Global    0.487473
Health    0.351675
Control   0.314296
MetaCog   0.108215
Others    0.158427
Hopeless  0.533973
OthViews  0.504708


In [49]:
output_psm

array([[ 0.72713769],
       [ 0.75510694],
       [ 0.57793988],
       [ 0.7528724 ],
       [ 0.27879326],
       [-0.01286433],
       [ 0.22367449],
       [ 0.62756154],
       [ 0.57866759]])

In [50]:
print('RNN Per-Schema Testset Output')
print(pd.DataFrame(data=output_psm,index=schemas,columns=['estimate']))

RNN Per-Schema Testset Output
          estimate
Attach    0.727138
Comp      0.755107
Global    0.577940
Health    0.752872
Control   0.278793
MetaCog  -0.012864
Others    0.223674
Hopeless  0.627562
OthViews  0.578668


In [51]:
def my_rnn_fixed(test_X,test_y,single):
    if single:
        gof,preds=predict_schema_psm(test_X,test_y,idx_psm[0])
    else:
        gof,preds=predict_schema_mlm(test_X,test_y,idx_mlm[0])
    return gof

In [52]:
output_psm_flat = [item for sublist in output_psm for item in sublist]
output_mlm_flat = [item for sublist in output_mlm for item in sublist]

In [53]:
print(f'Estimates of all models')
outputs = np.concatenate((output_kNN_class,output_kNN_reg,output_SVC, output_SVR, output_psm_flat, output_mlm_flat))
outputs=np.reshape(outputs,(9,6),order='F')
print(pd.DataFrame(data=outputs,index=schemas,columns=['kNN_class','kNN_reg','SVC','SVR','PSM','MLM']))

Estimates of all models
          kNN_class   kNN_reg       SVC       SVR       PSM       MLM
Attach     0.550705  0.626743  0.647714  0.675340  0.727138  0.686899
Comp       0.690230  0.663091  0.684661  0.640866  0.755107  0.662779
Global     0.401123  0.411444  0.357601  0.489372  0.577940  0.487473
Health     0.742217  0.534902  0.729181  0.349064  0.752872  0.351675
Control    0.107526  0.231541       NaN  0.310007  0.278793  0.314296
MetaCog         NaN  0.104785       NaN  0.114894 -0.012864  0.108215
Others     0.279105  0.243713       NaN  0.185827  0.223674  0.158427
Hopeless   0.484137  0.513825  0.489903  0.535979  0.627562  0.533973
OthViews   0.454565  0.458473  0.476297  0.516635  0.578668  0.504708


## Bootstrapping for confidence intervals
<br>
Since all models are expensive to run, we only do a small bootstrapping to obtain some insight into how confident we can be about the predictions.
<br>
<br>
******* Placing this section in the end since it is a bit time consuming to compute. Takes 3+ hours to complete. *******

In [54]:
# we adopt the algorithm from the following website:
# https://machinelearningmastery.com/calculate-bootstrap-confidence-intervals-machine-learning-results-python/
# def bootstrap
def bootstrap(iterations,sample_size, sample_embeds, sample_labels,classification,model):
    stats = np.zeros((iterations,9))
    for l in range(iterations):
        # prepare bootstrap sample
        bootstrap_sample_indx = random.sample(list(enumerate(sample_embeds)), sample_size)
        bootstrap_sample_utts = [sample_embeds[i] for (i,j) in bootstrap_sample_indx]
        bootstrap_sample_labels = [sample_labels[i] for (i,j) in bootstrap_sample_indx]
        # evaluate model
        if model=="knn":
            model_gof=my_kNN(np.array(bootstrap_sample_utts),np.array(bootstrap_sample_labels),classification)
        elif model=="svm":
            model_gof=my_svm(np.array(bootstrap_sample_utts),np.array(bootstrap_sample_labels),classification)
        elif model=="rnn":
            model_gof=my_rnn_fixed(np.array(bootstrap_sample_utts),np.array(bootstrap_sample_labels),classification)
        stats[l,:] = model_gof
    # confidence intervals
    cis = np.zeros((2,9))
    alpha = 0.95
    p = ((1.0-alpha)/2.0) * 100
    cis[0,:] = [max(0.0, np.percentile(stats[:,i], p)) for i in range(9)]
    p = (alpha+((1.0-alpha)/2.0)) * 100
    cis[1,:] = [min(1.0, np.percentile(stats[:,i], p)) for i in range(9)]
    return cis

# configure bootstrap
n_iterations = 100
n_size = int(len(val_text) * 0.75)

### Calculating confidence intervals for KNN


In [ ]:
%%time
# wall time to run: ~ 3h 30min
# bootstrap confidence intervals for kNN regression and classification
bs_knn_reg = bootstrap(n_iterations,n_size,test_embedutts,test_labels,0,"knn")
bs_knn_class = bootstrap(n_iterations,n_size,test_embedutts,test_labels,1,"knn")

In [ ]:
print(f'KNN Regression 95% Confidence Intervals')
print(pd.DataFrame(data=np.transpose(bs_knn_reg),index=schemas,columns=['low','high']))

In [ ]:
print(f'KNN Classification 95% Confidence Intervals')
print(pd.DataFrame(data=np.transpose(bs_knn_class),index=schemas,columns=['low','high']))

### Calculating confidence intervals for SVM


In [ ]:
%%time
# wall time to run: ~ 3min 15sec
# bootstrap confidence intervals for SVR and SVC
bs_svc = bootstrap(n_iterations,n_size,test_embedutts,test_labels,1,"svm")
bs_svr = bootstrap(n_iterations,n_size,test_embedutts,test_labels,0,"svm")

In [ ]:
print(f'SVM Classification 95% Confidence Intervals')
print(pd.DataFrame(data=np.transpose(bs_svc),index=schemas,columns=['low','high']))

In [ ]:
print(f'SVM Regression 95% Confidence Intervals')
print(pd.DataFrame(data=np.transpose(bs_svr),index=schemas,columns=['low','high']))

### Calculating confidence intervals for RNN


In [ ]:
%%time
# wall time to run: ~ 37min
#bootstrapping the 95% confidence intervals
bs_mlm = bootstrap(n_iterations,n_size,padded_test,test_labels,0,"rnn")
bs_psm = bootstrap(n_iterations,n_size,padded_test,test_labels,1,"rnn")

In [ ]:
print(f'Multilabel RNN Classification 95% Confidence Intervals')
print(pd.DataFrame(data=np.transpose(bs_mlm),index=schemas,columns=['low','high']))

In [ ]:
print(f'Per-Schema RNN Classification 95% Confidence Intervals')
print(pd.DataFrame(data=np.transpose(bs_psm),index=schemas,columns=['low','high']))

### Comparing condidence intervals for all three algorithms


In [ ]:
print(f'Lower CIs of all models')
lower_cis = np.concatenate((bs_knn_class[0],bs_knn_reg[0],bs_svc[0], bs_svr[0], bs_psm[0], bs_mlm[0]))
lower_cis=np.reshape(lower_cis,(9,6),order='F')
print(pd.DataFrame(data=lower_cis,index=schemas,columns=['kNN_class','kNN_reg','SVC','SVR','PSM','MLM']))

In [ ]:
print(f'Upper CIs of all models')
upper_cis = np.concatenate((bs_knn_class[1],bs_knn_reg[1],bs_svc[1], bs_svr[1], bs_psm[1], bs_mlm[1]))
upper_cis=np.reshape(upper_cis,(9,6),order='F')
print(pd.DataFrame(data=upper_cis,index=schemas,columns=['kNN_class','kNN_reg','SVC','SVR','PSM','MLM']))